# 🚀 Huấn luyện Chatbot AI trên Google Colab

Notebook này giúp bạn tận dụng GPU miễn phí của Google để huấn luyện model nhanh hơn.

### Hướng dẫn sử dụng:
1. Vào menu **Runtime** -> **Change runtime type** -> Chọn **T4 GPU**.
2. Upload file `train_data.csv` của bạn lên Colab (kéo thả vào mục Files bên trái).
3. Chạy lần lượt các bước bên dưới.

In [ ]:
# 1. Cài đặt thư viện cần thiết
!pip install transformers torch sentencepiece accelerate

In [ ]:
# 2. Import thư viện
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import os

# Cấu hình
MODEL_NAME = "google/mt5-small"
MAX_LEN = 128
EPOCHS = 5
BATCH_SIZE = 8  # Tăng lên 8 vì Colab GPU mạnh hơn máy cá nhân

In [ ]:
# 3. Chuẩn bị dữ liệu (Dataset Class)
class QADataset(Dataset):
    def __init__(self, tokenizer, data, max_len):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        question = str(row['question'])
        answer = str(row['answer'])

        # Format input cho T5: "question: ..."
        input_text = f"question: {question}"
        target_text = answer

        # Tokenize Input
        input_enc = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize Output (Label)
        target_enc = self.tokenizer(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        
        # Mask padding tokens in labels
        labels = target_enc.input_ids.flatten()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_enc.input_ids.flatten(),
            'attention_mask': input_enc.attention_mask.flatten(),
            'labels': labels
        }

In [ ]:
# 4. Bắt đầu Huấn luyện
def train_model():
    # Kiểm tra GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🔥 Đang chạy trên: {device}")

    # Load Tokenizer & Model
    print(f"📥 Đang tải model {MODEL_NAME}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    model.to(device)

    # Load Data
    if not os.path.exists('train_data.csv'):
        print("❌ Lỗi: Không tìm thấy file train_data.csv. Hãy upload lên Colab trước!")
        return
    
    print("📚 Đang đọc dữ liệu...")
    df = pd.read_csv('train_data.csv')
    dataset = QADataset(tokenizer, df, MAX_LEN)

    # Cấu hình Training
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        logging_dir='./logs',
        logging_steps=10,
        save_strategy="no",  # Không save checkpoint rác để tiết kiệm dung lượng
        report_to="none",
        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset
    )

    print("🏋️‍♂️ Bắt đầu huấn luyện...")
    trainer.train()

    # Lưu model cuối cùng
    print("💾 Đang lưu model...")
    save_path = "./my_generative_bot"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Hoàn tất! Model đã lưu tại: {save_path}")

train_model()

In [ ]:
# 5. Nén và Tải model về máy
!zip -r my_generative_bot.zip my_generative_bot
from google.colab import files
files.download('my_generative_bot.zip')